In [1]:
import os


In [2]:
%pwd

'/Users/aayushaggarwal/Desktop/Deep-learning-ml-dvc/research'

In [3]:
os.chdir('../')

In [4]:
%pwd

'/Users/aayushaggarwal/Desktop/Deep-learning-ml-dvc'

In [5]:
from pathlib import Path
from dataclasses import dataclass

@dataclass(frozen=True)
class ModelConfig:
    root_dir:Path
    model_path:Path
    updated_model_path:Path
    learning_rate: float
    image_size:list
    include_top:bool
    weights:str
    classes:int

In [6]:
from Classifier.constants import *
from Classifier.utils.common import read_yaml,create_directories

In [7]:
class ConfigurationManager:

    def __init__(self,
                 config_file_path=CONFIG_FILE_PATH,
                 params_file_path=PARAMS_FILE_PATH):
        self.config=read_yaml(config_file_path)
        self.params=read_yaml(params_file_path)

        create_directories([self.config.artifacts_root])
    
    def get_base_model_config(self)->ModelConfig:
        config = self.config.prepare_model
        params = self.params
        create_directories([config.root_dir])

        model_config=ModelConfig(
            root_dir=config.root_dir,
            model_path=config.model_path,
            updated_model_path=config.updated_model_path,
            learning_rate=params.LEARNING_RATE,
            image_size=params.IMAGE_SIZE,
            include_top=params.INCLUDE_TOP,
            weights = params.WEIGHTS,
            classes= params.CLASSES
            
        )
        return model_config
        

In [8]:
import os
import urllib.request as request
import tensorflow as tf
import keras 
from zipfile import ZipFile

In [9]:
class Prepare_Base_Model:
    def __init__(self,config:ModelConfig):
        self.config=config
    

    def get_base_model(self):
            self.model=keras.applications.vgg16.VGG16(
            input_shape=self.config.image_size,
            weights=self.config.weights,
            include_top=self.config.include_top)
            
            self.save_model(path=self.config.model_path, model=self.model)

    @staticmethod
    def prepare_full_model(model,classes,freeze_all,freeze_till,learning_rate):
        if freeze_all:
              model.trainable=False
            
        elif (freeze_till is not None) and (freeze_till>0):
             for layers in model.layers[:-freeze_till]:
                  model.trainable=False
        

        flatten_in=keras.layers.Flatten()(model.output)
        x= keras.layers.Dense(1000,activation='relu')(flatten_in)
        predictions= keras.layers.Dense(units=classes,
                                        activation='softmax',
                                        )(x)
        full_model = tf.keras.models.Model(
            inputs=model.input,
            outputs=predictions
        )

        full_model.compile(
            optimizer=keras.optimizers.legacy.Adam(learning_rate=learning_rate),
            loss=keras.losses.CategoricalCrossentropy(),
            metrics=["accuracy"]
        )
        full_model.summary()
        return full_model


    def update_base_model(self):
        self.full_model = self.prepare_full_model(
            model=self.model,
            classes=self.config.classes,
            freeze_all=True,
            freeze_till=None,
            learning_rate=self.config.learning_rate
        )

        self.save_model(path=self.config.updated_model_path, model=self.full_model)

    @staticmethod
    def save_model(path:Path,model:keras.Model):
         model.save(path)

In [10]:
try:
    config = ConfigurationManager()
    params_base_model_config = config.get_base_model_config()
    params_base_model = Prepare_Base_Model(config=params_base_model_config)
    params_base_model.get_base_model()
    params_base_model.update_base_model()

except Exception as e:
    raise e

[2024-09-16 14:50:41,158: INFO: common: yaml file: config/config.yaml loaded successfully]
[2024-09-16 14:50:41,161: INFO: common: yaml file: params.yaml loaded successfully]
[2024-09-16 14:50:41,161: INFO: common: created directory at: artifacts]
[2024-09-16 14:50:41,162: INFO: common: created directory at: artifacts/base_model]
[2024-09-16 14:50:41,400: WARNING: saving_utils: Compiled the loaded model, but the compiled metrics have yet to be built. `model.compile_metrics` will be empty until you train or evaluate the model.]
Model: "model"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_1 (InputLayer)        [(None, 224, 224, 3)]     0         
                                                                 
 block1_conv1 (Conv2D)       (None, 224, 224, 64)      1792      
                                                                 
 block1_conv2 (Conv2D)       (None, 224, 224, 64)      3

/Users/aayushaggarwal/anaconda3/envs/myenv/lib/python3.10/site-packages/keras/src/engine/training.py:3103: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(
